In [4]:
import os
import chess
import time
import chess.svg
import traceback
import base64
import import_ipynb
from BoardState import StateBoard

In [7]:
MAX_SCORE = 100000

class MCTSNode(object):
    
    def __init__(self, game_state, parent=None, move=None):
        self.game_state = game_state
        self.parent = parent
        self.move = move
        self.win_counts = {
            chess.BLACK: 0,
            chess.WHITE: 0,
        }
        self.num_rollouts = 0
        self.children = []
        self.unvisited_moves = game_state.board.legal_moves
        
    def can_add_child(self):
        return len(self.unvisited_moves) > 0

    def is_terminal(self):
        return self.game_state.board.is_game_over()

    def winning_frac(self, player):
        return float(self.win_counts[player]) / float(self.num_rollouts)
        
    def add_random_child(self):
        index = random.randint(0, len(self.unvisited_moves) - 1)
        new_move = self.unvisited_moves.pop(index)
        new_game_state = self.game_state.apply_move(new_move)
        new_node = MCTSNode(new_game_state, self, new_move)
        self.children.append(new_node)
        return new_node

    def record_win(self, winner):
        self.win_counts[winner] += 1
        self.num_rollouts += 1
        
class MCTSAgent():
    
    def uct_score(parent_rollouts, child_rollouts, win_pct, temperature):
        exploration = math.sqrt(math.log(parent_rollouts) / child_rollouts)
        return win_pct + temperature * exploration
    
    def select_child(self, node):
        total_rollouts = sum(child.num_rollouts for child in node.children)

        best_score = -1
        best_child = None
        for child in node.children:
            score = uct_score(
                total_rollouts,
                child.num_rollouts,
                child.winning_pct(node.game_state.next_player),
                self.temperature)
            if score > best_score:
                best_score = uct_score
                best_child = child
        return best_child
    
    def select_move(self, game_state):
        root = MCTSNode(game_state)

        for i in range(self.num_rounds):
            node = root
            while (not node.can_add_child()) and (not node.is_terminal()):
                node = self.select_child(node)

            if node.can_add_child():
                node = node.add_random_child()

            winner = self.simulate_random_game(node.game_state)

            while node is not None:
                node.record_win(winner)
                node = node.parent
                
        best_move = None
        best_pct = -1.0
        for child in root.children:
            child_pct = child.winning_pct(game_state.next_player)
            if child_pct > best_pct:
                best_pct = child_pct
                best_move = child.move
        return best_move

class MiniMax_TreeSearch():
    
    def __init__(self):
        
        self.reset_Count
        self.dict = {}
        self.pieceVals = {chess.PAWN: 1,
            chess.KNIGHT: 3,
            chess.BISHOP: 3,
            chess.ROOK: 5,
            chess.QUEEN: 9,
            chess.KING: 0}
        
    def __call__(self,game_state):
        
        self.count += 1
        key = game_state.feature_tuple()
        if key not in self.dict:
            self.dict[key] = self.best_result(game_state)
        return self.dict[key]
        
    def reset_Count(self):
        self.count = 0
    
    
    def boardVal(self,game_state):
        
        board = game_state.board
        board_val = 0

        if board.is_game_over():
            if board.result() == "1-0":
                return MAX_SCORE
            elif board.result() == "0-1":
                return -MAX_SCORE
            else:
                return 0
        
        piece_map = game_state.board.piece_map()
        for piece in piece_map:
            
            aVal = self.pieceVals[piece_map[piece].piece_type]
            if piece_map[piece].color == chess.BLACK:
                board_val -= aVal
            else:
                board_val += aVal
                
        reset = board.turn
        board.turn = chess.WHITE
        board_val += .1 * board.legal_moves.count()
        board.turn = chess.BLACK
        board_val -= .1 * board.legal_moves.count()
        board.turn = reset
                
        return board_val
            
    def alpha_beta_result(self,game_state, max_depth,best_black,best_white, eval_fn):
        
        if max_depth == 0:
            return eval_fn(game_state)

        best_so_far = -MAX_SCORE
        for candidate_move in game_state.moves():
            next_state = game_state.board.push(candidate_move)
            opponent_best_result = alpha_beta_result(
                next_state, max_depth - 1,
                best_black, best_white,
                eval_fn)
            our_result = -1 * opponent_best_result
            game_state.board.pop()

            if our_result > best_so_far:
                best_so_far = our_result
            if game_state.board.turn == chess.WHITE:
                if best_so_far > best_white:
                    best_white = best_so_far
                outcome_for_black = -1 * best_so_far
                if outcome_for_black < best_black:
                    return best_so_far
            elif game_state.board.turn == chess.BLACK:
                if best_so_far > best_black:
                    best_black = best_so_far
                outcome_for_white = -1 * best_so_far
                if outcome_for_white < best_white:
                    return best_so_far

        return best_so_far

